In [16]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sparentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)
sys.path.insert(0,sparentdir)

import datetime
import numpy as np
import scipy.stats as stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import create
import generate
import recall
import simulations
import simsave
import load_curves

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import matplotlib as mpl
import copy
import pickle
import numpy as np
import pandas as pd
import time
import seaborn as sns


N = 100
p = 0.5
c = 0.65
r = 0.1
M = 15

set_of_Ss = np.linspace(0,5.0,51)
set_of_Cs = np.linspace(0.1,0.4,13)
s_space = np.linspace(-10.0,15.0,10001)

In [17]:
#Get discretized posterior over S given a pattern and a connection matrix. 
def get_posterior_s(W, x, M, e_s2, frac, prior, s_space):
    
    N = len(x)
    count = (N * (N - 1)) / 2
    
    w_on = np.zeros(np.int(count))
    w_on[:np.int(count*frac)] = 1
    np.random.shuffle(w_on)

    mask = np.zeros((N, N))
    mask[np.triu_indices(N, 1)] = w_on
    W_f = mask * W

    total = 4*np.sum(W_f*(-1)**(x[:, None]+x))
    

    mu_W = total / (count*frac)
    var_W = (M - 1) * e_s2 / (count*frac)
    
    prob_s = stats.norm(mu_W, var_W).pdf(s_space) * prior
    
    return prob_s

In [18]:
#Store Results from Simulation

trials = 2000

recovered_s_control     = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
recovered_s_hlesion     = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])

recovered_s_control_lur = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
recovered_s_hlesion_lur = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])

error_known             = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
error_dual              = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
error_mono              = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])

proj_x_true             = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
proj_x_known            = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
proj_x_dual             = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])
proj_x_mono             = np.zeros([len(set_of_Cs), len(set_of_Ss), trials])

In [19]:
# Define the Background distribution
k_back = 1.0
mu_back = 1.0
background_dist = stats.gamma(k_back, 0, mu_back/k_back)

mu_P0 = mu_back
sigma_P0 = np.sqrt(mu_back*mu_back/k_back)
e_s2_P0 = mu_P0**2 + sigma_P0**2

In [24]:
update_s_known = create.update_s_known()
update_s_frac = create.update_s_frac(mu_P0, 10000, s_factor=1.0, beta=1000, frac=1.0)
frac = 1.0

flat = stats.norm(mu_P0, 10000).pdf
flat_v = flat(s_space) / np.sum(flat(s_space))
prior_s_v = flat_v

start_time = time.time()

for t in range(681, trials):
    print(str(t) + '; '+str((time.time() - start_time)/60))
    start_time = time.time()

    #Generate the background patterns and strengths
    xs, ss    = generate.memories_gaussian(N, p, M, mu_P0, sigma_P0)
    ss        = background_dist.rvs(N)

    xt = generate.noise(xs[0],r)

    for i_c, c in enumerate(set_of_Cs):

        update_x = create.update_x(lambda snr: c, e_s2 = e_s2_P0, beta=1000)
        
        for i_s, s in enumerate(set_of_Ss):

            ss[0] = s

            W = generate.weight_matrix(xs, ss, 0.5)
            proj_x_true[i_c, i_s, t] = np.sum(xs[0])

            s0 = update_s_frac(W, xt, M, mu_P0)/ np.max([0.15,(1-2*r)**2])

            x_samples, s_samples, _ = recall.maps(W, M, xt, s0, update_x, update_s_frac, samples = 100)
            x_recall                = np.median(x_samples[-50:],axis=0)
            
            post_s_v = get_posterior_s(W, x_recall, M, e_s2_P0, frac, prior_s_v, s_space)
            recovered_s_control[i_c, i_s, t] = s_space[np.argmax(post_s_v)]

            post_s_v = get_posterior_s(W, xt, M, e_s2_P0, frac, prior_s_v, s_space)
            recovered_s_hlesion[i_c, i_s, t] = s_space[np.argmax(post_s_v)]

            error_dual[i_c, i_s, t] = np.sum((xs[0]+x_recall)%2)
            proj_x_dual[i_c, i_s, t] = np.sum(x_recall)


            ### Mono

            x_samples, s_samples, _ = recall.maps(W, M, xt, mu_P0, update_x, update_s_known, samples = 100)
            x_recall                = np.median(x_samples[-50:],axis=0)

            error_mono[i_c, i_s, t] = np.sum((xs[0]+x_recall)%2)
            proj_x_mono[i_c, i_s, t] = np.sum(x_recall)

            ### Known

            x_samples, s_samples, _ = recall.maps(W, M, xt, ss[0], update_x, update_s_known, samples = 100)
            x_recall                = np.median(x_samples[-50:],axis=0)

            error_known[i_c, i_s, t] = np.sum((xs[0]+x_recall)%2)
            proj_x_known[i_c, i_s, t] = np.sum(x_recall)


681; 1.0581811269124349e-05
682; 0.2873009522755941
683; 0.28169025182724
684; 0.29086143573125206
685; 0.29586172898610436
686; 0.29078945716222127
687; 0.33822429974873863
688; 0.34440772930781044
689; 0.2927374720573425
690; 0.3191663424173991
691; 0.31406320333480836
692; 0.2975044846534729
693; 0.29338765144348145
694; 0.3302408536275228
695; 0.332452388604482
696; 0.29790846904118856
697; 0.3132623553276062
698; 0.29562456607818605
699; 0.3065841714541117
700; 0.3317107836405436
701; 0.2716930150985718
702; 0.2815412243207296
703; 0.28777734835942587
704; 0.26979516744613646
705; 0.3287623286247253
706; 0.2951267560323079
707; 0.36468073129653933
708; 0.2960902253786723
709; 0.30478291511535643
710; 0.3063162883122762
711; 0.29268109798431396
712; 0.29654783407847085
713; 0.2988940914471944
714; 0.3075528065363566
715; 0.34960366487503053
716; 0.2920417825380961
717; 0.29471325874328613
718; 0.29573275248209635
719; 0.3732595046361287
720; 0.2813388705253601
721; 0.28720418214797

In [25]:
np.save('./saves_recall/recovered_s_control.npy',recovered_s_control)
np.save('./saves_recall/recovered_s_hlesion.npy',recovered_s_hlesion)
np.save('./saves_recall/recovered_s_control_lur.npy',recovered_s_control_lur)
np.save('./saves_recall/recovered_s_hlesion_lur.npy',recovered_s_hlesion_lur)

np.save('./saves_recall/error_known.npy', error_known)
np.save('./saves_recall/error_dual.npy', error_dual)
np.save('./saves_recall/error_mono.npy', error_mono)

np.save('./saves_recall/proj_x_true.npy', proj_x_true)
np.save('./saves_recall/proj_x_known.npy', proj_x_known)
np.save('./saves_recall/proj_x_dual.npy', proj_x_dual)
np.save('./saves_recall/proj_x_mono.npy', proj_x_mono)